# 可见性和属性装饰器
在很多面向对象编程语言中，对象的属性通常会被设置为私有（private）或受保护（protected）的成员，简单的说就是不允许直接访问这些属性；对象的方法通常都是公开的（public），因为公开的方法是对象能够接受的消息，也是对象暴露给外界的调用接口，这就是所谓的访问可见性。在 Python 中，可以通过给对象属性名添加前缀下划线的方式来说明属性的访问可见性，例如，可以用__name表示一个私有属性，_name表示一个受保护属性，代码如下所示。

In [12]:
class Student:

    def __init__(self, name, age):
        self.__name = name
        self.__age = age

    def study(self, course_name):
        print(f'{self.__name}正在学习{course_name}.')


stu = Student('王大锤', 20)
stu.study('Python程序设计')
print(stu.__name)  # AttributeError: 'Student' object has no attribute '__name'

王大锤正在学习Python程序设计.


AttributeError: 'Student' object has no attribute '__name'

In [ ]:
# Use property to make the function become atrributes
class Triangle(object):
    """三角形"""

    def __init__(self, a, b, c):
        """初始化方法"""
        self.a = a
        self.b = b
        self.c = c

    @staticmethod
    def is_valid(a, b, c):
        """判断三条边长能否构成三角形(静态方法)"""
        return a + b > c and b + c > a and a + c > b

    @property
    def perimeter(self):
        """计算周长"""
        return self.a + self.b + self.c

    @property
    def area(self):
        """计算面积"""
        p = self.perimeter / 2
        return (p * (p - self.a) * (p - self.b) * (p - self.c)) ** 0.5


t = Triangle(3, 4, 5)
print(f'周长: {t.perimeter}')
print(f'面积: {t.area}')

周长: 12
面积: 6.0


简单起见，我们的扑克只有52张牌（没有大小王），游戏需要将 52 张牌发到 4 个玩家的手上，每个玩家手上有 13 张牌，按照黑桃、红心、草花、方块的顺序和点数从小到大排列，暂时不实现其他的功能。

In [16]:
from enum import Enum


class Suite(Enum):
    """花色(枚举)"""
    SPADE, HEART, CLUB, DIAMOND = range(4)


for suite in Suite:
    print(f'{suite}: {suite.value}')

class Card:
    """牌"""

    def __init__(self, suite, face):
        self.suite = suite
        self.face = face

    def __lt__(self, other):
        """定义小于运算符，用于排序"""
        if self.suite != other.suite:
            return self.suite.value < other.suite.value
        return self.face < other.face

    def __repr__(self):
        suites = '♠♥♣♦'
        faces = ['', 'A', '2', '3', '4', '5', '6', '7', '8', '9', '10', 'J', 'Q', 'K']
        return f'{suites[self.suite.value]}{faces[self.face]}'  # 返回牌的花色和点数


card1 = Card(Suite.SPADE, 5)
card2 = Card(Suite.HEART, 13)
print(card1)  # ♠5 
print(card2)  # ♥K


import random


class Poker:
    """扑克"""

    def __init__(self):
        self.cards = [Card(suite, face) 
                      for suite in Suite
                      for face in range(1, 14)]  # 52张牌构成的列表
        self.current = 0  # 记录发牌位置的属性

    def shuffle(self):
        """洗牌"""
        self.current = 0
        random.shuffle(self.cards)  # 通过random模块的shuffle函数实现随机乱序

    def deal(self):
        """发牌"""
        card = self.cards[self.current]
        self.current += 1
        return card

    @property
    def has_next(self):
        """还有没有牌可以发"""
        return self.current < len(self.cards)

poker = Poker()
print(poker.cards)  # 洗牌前的牌
poker.shuffle()
print(poker.cards)  # 洗牌后的牌

class Player:
    """玩家"""

    def __init__(self, name):
        self.name = name
        self.cards = []  # 玩家手上的牌

    def get_one(self, card):
        """摸牌"""
        self.cards.append(card)

    def arrange(self):
        """整理手上的牌"""
        self.cards.sort()


poker = Poker()
poker.shuffle()
players = [Player('东邪'), Player('西毒'), Player('南帝'), Player('北丐')]
# 将牌轮流发到每个玩家手上每人13张牌
for _ in range(13):
    for player in players:
        player.get_one(poker.deal())
# 玩家整理手上的牌输出名字和手牌
for player in players:
    player.arrange()
    print(f'{player.name}: ', end='')
    print(player.cards)

Suite.SPADE: 0
Suite.HEART: 1
Suite.CLUB: 2
Suite.DIAMOND: 3
♠5
♥K
[♠A, ♠2, ♠3, ♠4, ♠5, ♠6, ♠7, ♠8, ♠9, ♠10, ♠J, ♠Q, ♠K, ♥A, ♥2, ♥3, ♥4, ♥5, ♥6, ♥7, ♥8, ♥9, ♥10, ♥J, ♥Q, ♥K, ♣A, ♣2, ♣3, ♣4, ♣5, ♣6, ♣7, ♣8, ♣9, ♣10, ♣J, ♣Q, ♣K, ♦A, ♦2, ♦3, ♦4, ♦5, ♦6, ♦7, ♦8, ♦9, ♦10, ♦J, ♦Q, ♦K]
[♥K, ♥8, ♦10, ♥3, ♦3, ♠A, ♥2, ♦7, ♦8, ♠Q, ♥Q, ♠7, ♥A, ♦K, ♠2, ♣3, ♦A, ♦J, ♦5, ♠3, ♣7, ♠6, ♠J, ♠8, ♣5, ♥10, ♣10, ♣2, ♥7, ♣Q, ♣4, ♠5, ♦2, ♣J, ♣K, ♣9, ♥4, ♠9, ♥6, ♦9, ♣A, ♠4, ♣6, ♥J, ♥9, ♦Q, ♥5, ♠K, ♣8, ♦6, ♠10, ♦4]
东邪: [♠7, ♠8, ♠J, ♠Q, ♠K, ♥7, ♣6, ♣7, ♣9, ♦2, ♦3, ♦J, ♦Q]
西毒: [♠A, ♠2, ♠4, ♠5, ♠9, ♥A, ♥2, ♥3, ♥8, ♥10, ♥J, ♦7, ♦K]
南帝: [♠3, ♠6, ♥6, ♥9, ♥Q, ♣4, ♣5, ♣8, ♣10, ♣Q, ♣K, ♦5, ♦10]
北丐: [♠10, ♥4, ♥5, ♥K, ♣A, ♣2, ♣3, ♣J, ♦A, ♦4, ♦6, ♦8, ♦9]
